In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from datetime import datetime

In [ ]:
# Define your API key
API_KEY = "XXXXXXXXXX"

# Define your YouTube channel ID
CHANNEL_ID = "XXXXXXXXXX"

In [ ]:
# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

from dateutil import parser

def fetch_video_data(video_id):
    video_response = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=video_id
    ).execute()

    if video_response['items']:
        video = video_response['items'][0]
        snippet = video['snippet']
        statistics = video['statistics']
        content_details = video['contentDetails']

        duration = content_details.get('duration')

        # Convert ISO datetime string to a standard datetime object
        publish_datetime = parser.isoparse(snippet['publishedAt'])
        publish_date = publish_datetime.strftime("%Y-%m-%d")

        video_data = {
            'video_id': video_id,
            'title': snippet['title'],
            'upload_date': publish_date,
            'publishing_time': publish_datetime,
            'duration': duration if duration else None,
            'view_count': statistics['viewCount'],
            'like_count': statistics.get('likeCount', 0),
            'comment_count': statistics.get('commentCount', 0)
        }

        return video_data
    else:
        return None



# Function to fetch videos uploaded to the channel
def fetch_channel_videos(channel_id):
    videos = []
    next_page_token = None

    while True:
        playlist_response = youtube.search().list(
            part='snippet',
            channelId=channel_id,
            type='video',
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_response['items']:
            video_id = item['id']['videoId']
            video_data = fetch_video_data(video_id)
            if video_data:
                videos.append(video_data)

        next_page_token = playlist_response.get('nextPageToken')
        if not next_page_token:
            break

    return videos

# Fetch videos uploaded to the channel
channel_videos = fetch_channel_videos(CHANNEL_ID)

# Create a DataFrame
df = pd.DataFrame(channel_videos)

# Print the DataFrame
print(df)

        video_id                               title upload_date  \
0    p_PH5kD6CyM  SHALLIPOPI X COSTA TITCH TYPE BEAT  2023-12-15   
1    s9v_hMQSR8Y        DOUBLE (AFROBEATS TYPE BEAT)  2022-12-16   
2    a0MlTHduXkU          FOLD (AFROBEATS TYPE BEAT)  2021-12-10   
3    hnZNffTECms              ASA X PPRIME TYPE BEAT  2022-06-21   
4    o6TgwAE2CwU          KHAID X OMAH LAY TYPE BEAT  2023-12-09   
..           ...                                 ...         ...   
200  x5O5x5sRoGQ               FAVE X TENI TYPE BEAT  2022-02-11   
201  MpbweU3RN9M          (SMOOTH) FIREBOY TYPE BEAT  2021-04-17   
202  Xb5QSouAtQU         TRENT (AFROBEATS TYPE BEAT)  2021-11-12   
203  -g_9TzgD6VI                    JOEBOY TYPE BEAT  2020-10-23   
204  CHO1aBSK2UI                YOUNG JONN TYPE BEAT  2022-12-02   

              publishing_time duration view_count like_count comment_count  
0   2023-12-15 05:48:04+00:00  PT1M16S       4745         37             0  
1   2022-12-16 09:08:05+00:00

In [ ]:

# Define a function to populate the DataFrame
def get_videos(df):
    # Iterate over your video data and append rows to the DataFrame
    for video in channel_videos:
        df = df._append({
            "video_duration": video['duration'],
            "publishing_time": video['publishing_time'],
            "view_count": video['view_count'],
            "like_count": video['like_count'],
            "comment_count": video['comment_count'],
            "video_id": video['video_id'],
            "video_title": video['title'],
            "upload_date": video['upload_date']
        }, ignore_index=True)
    return df

# Create an empty DataFrame with specified column names
df2 = pd.DataFrame(columns=["video_duration", "publishing_time", "view_count", "like_count", "comment_count", "video_id", "video_title", "upload_date"])

# Populate the DataFrame with video data
df2 = get_videos(df2)



# Display the DataFrame
print(df2)

    video_duration           publishing_time view_count like_count  \
0          PT1M16S 2023-12-15 05:48:04+00:00       4745         37   
1          PT1M27S 2022-12-16 09:08:05+00:00        174          5   
2          PT1M16S 2021-12-10 05:22:05+00:00        279          4   
3          PT1M22S 2022-06-21 11:24:48+00:00        184          4   
4           PT2M7S 2023-12-09 08:37:10+00:00       6626         68   
..             ...                       ...        ...        ...   
200        PT1M19S 2022-02-11 16:25:30+00:00        140          1   
201        PT1M21S 2021-04-17 10:41:48+00:00         78          5   
202         PT3M5S 2021-11-12 17:07:51+00:00        131          2   
203        PT3M21S 2020-10-23 08:31:54+00:00         55          2   
204        PT1M27S 2022-12-02 06:03:31+00:00        327          9   

    comment_count     video_id                         video_title upload_date  
0               0  p_PH5kD6CyM  SHALLIPOPI X COSTA TITCH TYPE BEAT  2023-12-15

In [ ]:

# Function to convert ISO 8601 duration format to seconds
def duration_to_seconds(duration):
    # Split the duration string into components (e.g., PT1M16S -> ['PT', '1M', '16S'])
    components = duration.split('PT')[1]
    minutes, seconds = 0, 0

    # Extract minutes if available
    if 'M' in components:
        minutes = int(components.split('M')[0])
        components = components.split('M')[1]

    # Extract seconds if available
    if 'S' in components:
        seconds = int(components.split('S')[0])

    # Convert to total seconds
    total_seconds = minutes * 60 + seconds
    return total_seconds

# Apply the function to the video_duration column and overwrite its content
df2['video_duration'] = df2['video_duration'].apply(duration_to_seconds)

# Display the modified DataFrame
print(df2)

     video_duration           publishing_time view_count like_count  \
0                76 2023-12-15 05:48:04+00:00       4745         37   
1                87 2022-12-16 09:08:05+00:00        174          5   
2                76 2021-12-10 05:22:05+00:00        279          4   
3                82 2022-06-21 11:24:48+00:00        184          4   
4               127 2023-12-09 08:37:10+00:00       6626         68   
..              ...                       ...        ...        ...   
200              79 2022-02-11 16:25:30+00:00        140          1   
201              81 2021-04-17 10:41:48+00:00         78          5   
202             185 2021-11-12 17:07:51+00:00        131          2   
203             201 2020-10-23 08:31:54+00:00         55          2   
204              87 2022-12-02 06:03:31+00:00        327          9   

    comment_count     video_id                         video_title upload_date  
0               0  p_PH5kD6CyM  SHALLIPOPI X COSTA TITCH TYPE BEAT

In [ ]:
df2.head(11)

,video_duration,publishing_time,view_count,like_count,comment_count,video_id,video_title,upload_date
0,76,2023-12-15 05:48:04+00:00,4745,37,0,p_PH5kD6CyM,SHALLIPOPI X COSTA TITCH TYPE BEAT,2023-12-15
1,87,2022-12-16 09:08:05+00:00,174,5,2,s9v_hMQSR8Y,DOUBLE (AFROBEATS TYPE BEAT),2022-12-16
2,76,2021-12-10 05:22:05+00:00,279,4,0,a0MlTHduXkU,FOLD (AFROBEATS TYPE BEAT),2021-12-10
3,82,2022-06-21 11:24:48+00:00,184,4,0,hnZNffTECms,ASA X PPRIME TYPE BEAT,2022-06-21
4,127,2023-12-09 08:37:10+00:00,6626,68,3,o6TgwAE2CwU,KHAID X OMAH LAY TYPE BEAT,2023-12-09
5,88,2023-10-06 04:20:22+00:00,152,5,3,e1pPfuu-GF8,REMA X SELENA GOMEZ TYPE BEAT,2023-10-06
6,129,2022-11-25 05:02:18+00:00,9896,101,6,7EH2ZfeUYpE,T.I BLAZE TYPE BEAT,2022-11-25
7,114,2021-06-18 12:07:38+00:00,11867,143,10,yLaeUvT2Nxs,NAIRA MARLEY X MOHBAD TYPE BEAT,2021-06-18
8,97,2022-05-07 11:32:02+00:00,12220,121,5,_deCvU0tCIA,KHAID X REMA TYPE BEAT,2022-05-07
9,66,2022-10-13 20:48:42+00:00,76,4,0,l2zCKhyslmg,WEIRD (AFROBEATS TYPE BEAT),2022-10-13


In [ ]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [ ]:

def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn


def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(205) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    video_duration INTEGER NOT NULL,
                    publishing_time TIME NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, video_duration, publishing_time, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        video_duration, publishing_time, view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, video_duration, publishing_time, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, video_duration, publishing_time, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                video_duration = %s,
                publishing_time = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, video_duration, publishing_time, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df2.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['video_duration'],
                          row['publishing_time'], row['view_count'], row['like_count'], row['comment_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'video_duration',
                                   'publishing_time', 'view_count', 'like_count', 'comment_count'])
    for i, row in df2.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['video_duration'],row['publishing_time'],
                       row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df._append(row)

    return tmp_df

In [ ]:

#Main

#database credentials
host_name = 'XXXXXXXXXX'
dbname = 'XXXXXXXXXX'
port = 'XXXXXXXXXX'
username = 'XXXXXXXXXX'
password = 'XXXXXXXXXX'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [ ]:
#create table

create_table(curr)

In [ ]:

#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

In [ ]:

#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:

#view data in db table

curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('p_PH5kD6CyM', 'SHALLIPOPI X COSTA TITCH TYPE BEAT', datetime.date(2023, 12, 15), 76, datetime.time(5, 48, 4), 4745, 37, 0), ('s9v_hMQSR8Y', 'DOUBLE (AFROBEATS TYPE BEAT)', datetime.date(2022, 12, 16), 87, datetime.time(9, 8, 5), 174, 5, 2), ('a0MlTHduXkU', 'FOLD (AFROBEATS TYPE BEAT)', datetime.date(2021, 12, 10), 76, datetime.time(5, 22, 5), 279, 4, 0), ('hnZNffTECms', 'ASA X PPRIME TYPE BEAT', datetime.date(2022, 6, 21), 82, datetime.time(11, 24, 48), 184, 4, 0), ('o6TgwAE2CwU', 'KHAID X OMAH LAY TYPE BEAT', datetime.date(2023, 12, 9), 127, datetime.time(8, 37, 10), 6626, 68, 3), ('e1pPfuu-GF8', 'REMA X SELENA GOMEZ TYPE BEAT', datetime.date(2023, 10, 6), 88, datetime.time(4, 20, 22), 152, 5, 3), ('7EH2ZfeUYpE', 'T.I BLAZE TYPE BEAT', datetime.date(2022, 11, 25), 129, datetime.time(5, 2, 18), 9896, 101, 6), ('yLaeUvT2Nxs', 'NAIRA MARLEY X MOHBAD TYPE BEAT', datetime.date(2021, 6, 18), 114, datetime.time(12, 7, 38), 11867, 143, 10), ('_deCvU0tCIA', 'KHAID X REMA TYPE BEAT', datetim

In [ ]:

def fetch_data_to_df(conn, query):
    # Execute the query
    with conn.cursor() as cur:
        cur.execute(query)
        # Fetch all rows from the result set
        rows = cur.fetchall()

    # Get column names from cursor description
    col_names = [desc[0] for desc in cur.description]

    # Create DataFrame
    df = pd.DataFrame(rows, columns=col_names)

    return df

# Query to fetch data
query = "SELECT * FROM VIDEOS"
# Fetch data into a DataFrame
df_from_db = fetch_data_to_df(conn, query)

# Close the connection
conn.close()

# Display the DataFrame
print(df_from_db)

        video_id                         video_title upload_date  \
0    p_PH5kD6CyM  SHALLIPOPI X COSTA TITCH TYPE BEAT  2023-12-15   
1    s9v_hMQSR8Y        DOUBLE (AFROBEATS TYPE BEAT)  2022-12-16   
2    a0MlTHduXkU          FOLD (AFROBEATS TYPE BEAT)  2021-12-10   
3    hnZNffTECms              ASA X PPRIME TYPE BEAT  2022-06-21   
4    o6TgwAE2CwU          KHAID X OMAH LAY TYPE BEAT  2023-12-09   
..           ...                                 ...         ...   
200  x5O5x5sRoGQ               FAVE X TENI TYPE BEAT  2022-02-11   
201  MpbweU3RN9M          (SMOOTH) FIREBOY TYPE BEAT  2021-04-17   
202  Xb5QSouAtQU         TRENT (AFROBEATS TYPE BEAT)  2021-11-12   
203  -g_9TzgD6VI                    JOEBOY TYPE BEAT  2020-10-23   
204  CHO1aBSK2UI                YOUNG JONN TYPE BEAT  2022-12-02   

     video_duration publishing_time  view_count  like_count  comment_count  
0                76        05:48:04        4745          37              0  
1                87        09

In [ ]:
df_from_db.head(10)

,video_id,video_title,upload_date,video_duration,publishing_time,view_count,like_count,comment_count
0,p_PH5kD6CyM,SHALLIPOPI X COSTA TITCH TYPE BEAT,2023-12-15,76,05:48:04,4745,37,0
1,s9v_hMQSR8Y,DOUBLE (AFROBEATS TYPE BEAT),2022-12-16,87,09:08:05,174,5,2
2,a0MlTHduXkU,FOLD (AFROBEATS TYPE BEAT),2021-12-10,76,05:22:05,279,4,0
3,hnZNffTECms,ASA X PPRIME TYPE BEAT,2022-06-21,82,11:24:48,184,4,0
4,o6TgwAE2CwU,KHAID X OMAH LAY TYPE BEAT,2023-12-09,127,08:37:10,6626,68,3
5,e1pPfuu-GF8,REMA X SELENA GOMEZ TYPE BEAT,2023-10-06,88,04:20:22,152,5,3
6,7EH2ZfeUYpE,T.I BLAZE TYPE BEAT,2022-11-25,129,05:02:18,9896,101,6
7,yLaeUvT2Nxs,NAIRA MARLEY X MOHBAD TYPE BEAT,2021-06-18,114,12:07:38,11867,143,10
8,_deCvU0tCIA,KHAID X REMA TYPE BEAT,2022-05-07,97,11:32:02,12220,121,5
9,l2zCKhyslmg,WEIRD (AFROBEATS TYPE BEAT),2022-10-13,66,20:48:42,76,4,0
